In [1]:
import json
import pandas as pd

article_list = []
with open("arxiv-metadata-oai-snapshot.json", "r") as f:
    for l in f:
        d = json.loads(l)
        article_id = d["id"]
        
        article_list.append(
            {
                "ID": article_id,
                "title": d["title"],
                "categories": d["categories"],
                "versions": d["versions"],
            }
        )


article_df = pd.DataFrame.from_records(article_list)  

In [2]:
article_df.head(10)

,ID,title,categories,versions
0,0704.0001,Calculation of prompt diphoton production cros...,hep-ph,"[{'version': 'v1', 'created': 'Mon, 2 Apr 2007..."
1,0704.0002,Sparsity-certifying Graph Decompositions,math.CO cs.CG,"[{'version': 'v1', 'created': 'Sat, 31 Mar 200..."
2,0704.0003,The evolution of the Earth-Moon system based o...,physics.gen-ph,"[{'version': 'v1', 'created': 'Sun, 1 Apr 2007..."
3,0704.0004,A determinant of Stirling cycle numbers counts...,math.CO,"[{'version': 'v1', 'created': 'Sat, 31 Mar 200..."
4,0704.0005,From dyadic $\Lambda_{\alpha}$ to $\Lambda_{\a...,math.CA math.FA,"[{'version': 'v1', 'created': 'Mon, 2 Apr 2007..."
5,0704.0006,Bosonic characters of atomic Cooper pairs acro...,cond-mat.mes-hall,"[{'version': 'v1', 'created': 'Sat, 31 Mar 200..."
6,0704.0007,Polymer Quantum Mechanics and its Continuum Limit,gr-qc,"[{'version': 'v1', 'created': 'Sat, 31 Mar 200..."
7,0704.0008,Numerical solution of shock and ramp compressi...,cond-mat.mtrl-sci,"[{'version': 'v1', 'created': 'Sat, 31 Mar 200..."
8,0704.0009,"The Spitzer c2d Survey of Large, Nearby, Inste...",astro-ph,"[{'version': 'v1', 'created': 'Mon, 2 Apr 2007..."
9,0704.0010,"Partial cubes: structures, characterizations, ...",math.CO,"[{'version': 'v1', 'created': 'Sat, 31 Mar 200..."


In [3]:
def filter_cs_articles(article_df: pd.DataFrame, start_year: int, end_year: int) -> pd.DataFrame:
    cs_articles = article_df[article_df['categories'].str.contains('^cs', regex=True)]

    # Extract the version and created date for the first version
    cs_articles[['version', 'created']] = cs_articles['versions'].apply(
        lambda x: pd.Series([x[0]['version'], x[0]['created']])
    )

    cs_articles['year'] = pd.to_datetime(
        cs_articles['created'], format='%a, %d %b %Y %H:%M:%S GMT'
    ).dt.year

    cs_articles = cs_articles[
        (cs_articles["year"] >= start_year) & (cs_articles["year"] <= end_year)
    ]

    cs_articles = cs_articles.drop(
        columns=["versions", "version", "created"], axis=1
    )

    return cs_articles

cs_articles = filter_cs_articles(article_df, 2018, 2022)

cs_articles.head()
cs_articles.info()


/Users/tapetrova/miniconda3/envs/tensorflow/lib/python3.6/site-packages/pandas/core/frame.py:3069: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
/Users/tapetrova/miniconda3/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


<class 'pandas.core.frame.DataFrame'>
Int64Index: 259577 entries, 929308 to 1795204
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   ID          259577 non-null  object
 1   title       259577 non-null  object
 2   categories  259577 non-null  object
 3   year        259577 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 9.9+ MB


In [4]:
cs_articles.head()

,ID,title,categories,year
929308,1801.00377,Help Me Find a Job: A Graph-based Approach for...,cs.IR cs.SI,2018
929315,1801.00384,Error-Robust Multi-View Clustering,cs.LG,2018
929316,1801.00385,Interactive Co-Design of Form and Function for...,cs.RO,2018
929318,1801.00387,Diversity Analysis of Millimeter-Wave Massive ...,cs.IT math.IT,2018
929319,1801.00388,Beyond Word Embeddings: Learning Entity and Co...,cs.CL cs.AI cs.IR cs.SI,2018


In [5]:
cs_articles.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 259577 entries, 929308 to 1795204
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   ID          259577 non-null  object
 1   title       259577 non-null  object
 2   categories  259577 non-null  object
 3   year        259577 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 9.9+ MB


In [6]:
cs_articles.to_csv('cs_articles.csv', sep = ",")